In [7]:
!pip install folium requests ipywidgets networkx geopy


In [8]:
import folium
import requests
import math
import networkx as nx
from geopy.geocoders import Nominatim
from ipywidgets import Text, FloatText, Button, VBox, HBox
from IPython.display import display, clear_output


In [9]:
# ---------------- CONFIG ----------------
TOMTOM_API_KEY = "UdQFxUibHzEsShySuY96wlJAK6BhzJvX"
OCM_API_KEY = "931d4ae3-014a-4ac1-8c4c-d1aa244c42de"
geolocator = Nominatim(user_agent="ev_route_app")

# ---------------- Helpers ----------------
def haversine(a, b):
    R = 6371.0
    lat1, lon1 = math.radians(a[0]), math.radians(a[1])
    lat2, lon2 = math.radians(b[0]), math.radians(b[1])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    aa = math.sin(dlat/2)**2 + math.cos(lat1)*math.cos(lat2)*math.sin(dlon/2)**2
    return R * 2 * math.atan2(math.sqrt(aa), math.sqrt(1-aa))

def parse_location(value):
    try:
        if "," in value:
            parts = list(map(float, value.split(",")))
            if len(parts) == 2:
                return parts
    except:
        pass
    loc = geolocator.geocode(value, timeout=10)
    if loc:
        return [loc.latitude, loc.longitude]
    raise ValueError(f"Could not resolve location: {value}")


In [13]:
def get_tomtom_routes(start, end):
    """
    Fetch TomTom fastest route (minimal version for Colab)
    Start/end: [lat, lon]
    """
    url = f"https://api.tomtom.com/routing/1/calculateRoute/{start[0]},{start[1]}:{end[0]},{end[1]}/json"
    params = {
        "key": TOMTOM_API_KEY,
        "routeType": "fastest"
    }
    r = requests.get(url, params=params, timeout=10)
    r.raise_for_status()
    data = r.json()
    routes = []
    for route in data.get("routes", []):
        coords = [[point["latitude"], point["longitude"]]
                  for leg in route["legs"]
                  for point in leg["points"]]
        dist = route["summary"]["lengthInMeters"]
        dur = route["summary"]["travelTimeInSeconds"]
        routes.append({"coords": coords, "distance": dist, "duration": dur})
    if not routes:
        raise ValueError("No route found from TomTom")
    return routes

#Test fetch
start = [17.6935526, 83.2921297]
end = [17.360589, 78.4740613]
routes = get_tomtom_routes(start, end)
print("Route points:", len(routes[0]['coords']), "Distance:", routes[0]['distance'])


Route points: 9259 Distance: 635993


In [15]:
def build_graph(routes, step=10):
    G = nx.Graph()
    coords = {}
    idx = 0
    for r in routes:
        pts = r["coords"][::step]
        for i in range(len(pts)-1):
            a,b = pts[i], pts[i+1]
            dist = haversine(a,b)*1000
            coords[idx] = a
            coords[idx+1] = b
            G.add_edge(idx, idx+1, weight=dist)
            idx += 1
    return G, coords

def a_star(G, start, goal, coords):
    def h(u, v):  # ✅ must take 2 arguments
        return haversine(coords[u], coords[goal])
    return nx.astar_path(G, start, goal, heuristic=h, weight="weight")

def cfp(G, start, goal):
    return nx.shortest_path(G, source=start, target=goal, weight="weight")

# Test graph
# G, coords = build_graph(routes)
# start_n, end_n = 0, max(coords.keys())
# path = a_star(G, start_n, end_n, coords)
# print("Path length:", len(path))


In [22]:
import branca.colormap as cm

# ---------------- Cache for OCM ----------------
charging_cache = {}  # key = (rounded lat, lon)

def find_nearby_charging_cached(lat, lon, max_distance_km=5):
    key = (round(lat, 5), round(lon,5))
    if key in charging_cache:
        return charging_cache[key]
    try:
        stations = find_nearby_charging(lat, lon, max_distance_km)
        charging_cache[key] = stations
        time.sleep(0.2)
        return stations
    except Exception as e:
        print("⚠️ Charging API request failed:", e)
        return []

# ---------------- UI ----------------
start_box = Text(value="Visakhapatnam", description="Start:")
end_box = Text(value="Hyderabad", description="End:")
charge_box = FloatText(value=100, description="Charge %:")
capacity_box = FloatText(value=50, description="Battery (kWh):")
range_box = FloatText(value=300, description="Range (km):")
btn = Button(description="Find Routes", button_style="success")

def on_click(_):
    clear_output(wait=True)
    display(ui)

    # Input parsing
    try:
        start = parse_location(start_box.value.strip())
        end = parse_location(end_box.value.strip())
        charge = charge_box.value
        full_range = range_box.value
    except Exception as e:
        print("❌ Error:", e)
        return

    # Fetch routes
    try:
        routes = get_tomtom_routes(start,end)
        if len(routes) == 1:
            routes.append({
                "coords": routes[0]["coords"][::2],
                "distance": routes[0]["distance"]*1.05,
                "duration": routes[0]["duration"]*1.05
            })
    except Exception as e:
        print("❌ Route fetch failed:", e)
        return

    # Build graph and coordinates
    coords_list_all = []
    for r in routes:
        G, coords = build_graph([r])
        coords_list_all.append([coords[n] for n in G.nodes])

    # Select best route (shortest distance)
    route_distances = [sum(haversine(coords_list[i], coords_list[i+1])
                           for i in range(len(coords_list)-1))
                       for coords_list in coords_list_all]
    best_index = route_distances.index(min(route_distances))
    best_coords_list = coords_list_all[best_index]

    # ---------------- Map ----------------
    m = folium.Map(location=start, zoom_start=6)

    # Draw alternative routes (blue dashed)
    for i, coords_list in enumerate(coords_list_all):
        if i != best_index:
            folium.PolyLine(coords_list, color="blue", weight=2, dash_array="5,10", popup="Alternative Route").add_to(m)

    # Start & Destination markers
    folium.Marker(best_coords_list[0], popup="Start 🟢", icon=folium.Icon(color="green", icon="play")).add_to(m)
    folium.Marker(best_coords_list[-1], popup="Destination 🔵", icon=folium.Icon(color="blue", icon="flag")).add_to(m)

    # ---------------- Battery Gradient ----------------
    remaining_range = (charge/100)*full_range
    dist_traveled = 0
    battery_remaining = charge
    required_charging_stops = []

    # Color map: red -> yellow -> green
    colormap = cm.LinearColormap(['red', 'yellow', 'green'], vmin=0, vmax=100)

    for i in range(len(best_coords_list)-1):
        seg = haversine(best_coords_list[i], best_coords_list[i+1])
        dist_traveled += seg
        battery_remaining -= (seg/full_range)*100

        # Draw segment with battery color
        seg_color = colormap(max(battery_remaining,0))
        folium.PolyLine([best_coords_list[i], best_coords_list[i+1]], color=seg_color, weight=6).add_to(m)

        # Check if charging required
        if dist_traveled >= remaining_range:
            stop_point = best_coords_list[i]
            stations = find_nearby_charging_cached(stop_point[0], stop_point[1])
            if stations:
                nearest = min(stations, key=lambda s: haversine(stop_point, s["coords"]))
                required_charging_stops.append({
                    "name": nearest["name"],
                    "coords": nearest["coords"],
                    "dist_traveled": dist_traveled,
                    "battery_remaining": max(battery_remaining,0)
                })
            else:
                required_charging_stops.append({
                    "name": "No Station Found",
                    "coords": stop_point,
                    "dist_traveled": dist_traveled,
                    "battery_remaining": max(battery_remaining,0)
                })
            remaining_range = full_range
            dist_traveled = 0
            battery_remaining = 100

    # ---------------- Plot Required Charging (Red) ----------------
    for stop in required_charging_stops:
        folium.Marker(
            stop["coords"],
            popup=f"🔴 Required Charging Stop: {stop['name']}<br>"
                  f"Distance traveled: {stop['dist_traveled']:.1f} km<br>"
                  f"Battery remaining: {stop['battery_remaining']:.1f}%",
            icon=folium.Icon(color="red")
        ).add_to(m)

    # ---------------- Plot Nearby Stations Along Route (Orange) ----------------
    for i in range(0,len(best_coords_list),10):  # fewer points to reduce API calls
        stations = find_nearby_charging_cached(best_coords_list[i][0], best_coords_list[i][1])
        for s in stations:
            folium.Marker(
                s["coords"],
                popup=f"🟠 Nearby Charging Station: {s['name']}",
                icon=folium.Icon(color="orange")
            ).add_to(m)

    # Add legend for battery
    colormap.caption = 'Battery % along route'
    colormap.add_to(m)

    display(m)

ui = VBox([HBox([start_box,end_box]), HBox([charge_box,capacity_box,range_box]), btn])
btn.on_click(on_click)
display(ui)
